# Deep Learning with PyTorch

This notebook introduces PyTorch a framework for building and training neural networks. PyTorch behaves like the arrays from Numpy. These numpy arrays are after all just tensors. PyTorch takes these tensors and makes it simple to move them to GPUs for the faster processing needed when training neural networks. It also provides a module that automatically calculates gradients ( for backpropagation) and another module specifically for building neural networks. All together, PyTorch ends up being more coherent with Python and the Numpy/Scipy stack compared to TensorFlow and other frameworks.

# Neural Networks

Deep Learning is based on artificial neural networks which have been around in some form since the late 1950s. The networks are built from individual parts approximating neurons, typically called units or simply "neurons." Each unit has some number of weighted inputs. These weighted inputs are summed together (a linear combination) then passed through an activation function to get the unit's output.

<img src="assets/simple_neuron.png" width=400px>

Mathematically this looks like: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i \right)
\end{align}
$$

With vectors this is the dot/inner product of two vectors:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$


### Stack them up!

We can assemble these unit neurons into layers and stacks, into a network of neurons. The output of one layer of neurons becomes the input for the next layer. With multiple input units and output units, we now need to express the weights as a matrix.

<img src='assets/multilayer_diagram_weights.png' width=450px>

We can express this mathematically with matrices again and use matrix multiplication to get linear combinations for each unit in one operation. For example, the hidden layer ($h_1$ and $h_2$ here) can be calculated 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

The output for this small network is found by treating the hidden layer as inputs for the output unit. The network output is expressed simply

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

## Tensors

It turns out neural network computations are just a bunch of linear algebra operations on *tensors*, a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor (RGB color images for example). The fundamental data structure for neural networks are tensors and PyTorch (as well as pretty much every other deep learning framework) is built around tensors.

<img src="assets/tensor_examples.svg" width=600px>

With the basics covered, it's time to explore how we can use PyTorch to build a simple neural network.

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch



Bad key "text.kerning_factor" on line 4 in
C:\Users\ismiguzel\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


First, let's see how we work with PyTorch tensors. These are the fundamental data structures of neural networks and PyTorch, so it's imporatant to understand how these work.

In [4]:
x = torch.rand(3,2)
x

tensor([[0.9164, 0.0367],
        [0.1069, 0.0427],
        [0.4550, 0.9483]])

In [5]:
y = torch.ones(x.size())
y

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [6]:
z = x + y
z

tensor([[1.9164, 1.0367],
        [1.1069, 1.0427],
        [1.4550, 1.9483]])

In general PyTorch tensors behave similar to Numpy arrays. They are zero indexed and support slicing.

In [7]:
z[0]

tensor([1.9164, 1.0367])

In [8]:
z[:, 1]

tensor([1.0367, 1.0427, 1.9483])

Tensors typically have two forms of methods, one method that returns another tensor and another method that performs the operation in place. That is, the values in memory for that tensor are changed without creating a new tensor. In-place functions are always followed by an underscore, for example `z.add()` and `z.add_()`.

In [9]:
# Return a new tensor z + 1
z.add(1)

tensor([[2.9164, 2.0367],
        [2.1069, 2.0427],
        [2.4550, 2.9483]])

In [11]:
# z tensor is unchanged
z

tensor([[1.9164, 1.0367],
        [1.1069, 1.0427],
        [1.4550, 1.9483]])

In [12]:
# Add 1 and update z tensor in-place
z.add_(1)

tensor([[2.9164, 2.0367],
        [2.1069, 2.0427],
        [2.4550, 2.9483]])

In [13]:
# z has been updated
z

tensor([[2.9164, 2.0367],
        [2.1069, 2.0427],
        [2.4550, 2.9483]])

### Reshaping

Reshaping tensors is a really common operation. First to get the size and shape of a tensor use `.size()`. Then, to reshape a tensor, use `.resize_()`. Notice the underscore, reshaping is an in-place operation.

In [14]:
z.size()

torch.Size([3, 2])

In [21]:
z.resize_(2, 3)

tensor([[2.9164, 2.0367, 2.1069],
        [2.0427, 2.4550, 2.9483]])

In [22]:
z

tensor([[2.9164, 2.0367, 2.1069],
        [2.0427, 2.4550, 2.9483]])

## Numpy to Torch and back

Converting between Numpy arrays and Torch tensors is super simple and useful. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [23]:
a = np.random.rand(4,3)
a

array([[0.21471945, 0.59318434, 0.92480832],
       [0.75766395, 0.42159667, 0.48488198],
       [0.22218143, 0.57428471, 0.1452528 ],
       [0.20156126, 0.02688476, 0.51469502]])

In [24]:
b = torch.from_numpy(a)
b

tensor([[0.2147, 0.5932, 0.9248],
        [0.7577, 0.4216, 0.4849],
        [0.2222, 0.5743, 0.1453],
        [0.2016, 0.0269, 0.5147]], dtype=torch.float64)

In [25]:
b.numpy()

array([[0.21471945, 0.59318434, 0.92480832],
       [0.75766395, 0.42159667, 0.48488198],
       [0.22218143, 0.57428471, 0.1452528 ],
       [0.20156126, 0.02688476, 0.51469502]])

The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [26]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

tensor([[0.4294, 1.1864, 1.8496],
        [1.5153, 0.8432, 0.9698],
        [0.4444, 1.1486, 0.2905],
        [0.4031, 0.0538, 1.0294]], dtype=torch.float64)

In [27]:
# Numpy array matches new values from Tensor
a

array([[0.4294389 , 1.18636868, 1.84961664],
       [1.51532791, 0.84319334, 0.96976396],
       [0.44436285, 1.14856943, 0.2905056 ],
       [0.40312253, 0.05376952, 1.02939004]])